In [1]:
import keras

Using TensorFlow backend.
/users/avsec/bin/anaconda3/envs/factornet/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from keras.models import Model

In [3]:
ls

bigwig.txt  dataloader_files/  feature.txt  model_files/  Untitled.ipynb
chip.txt    dataloader.py      meta.txt     other_files/


In [4]:
cd model_files

/srv/scratch/avsec/workspace/FactorNet/kipoi/CTCF/metaGENCODE_RNAseq_Unique35_DGF/model_files


In [5]:
ls

best_model.hdf5  model.json


In [6]:
import json

In [13]:
Model.from_config(json.load(open("model.json")))

KeyError: 'layers'

In [15]:
from keras.models import model_from_json

In [35]:
def input_layers(model):
    """Show all the input nodes of the network"""
    return [(l.name, l.input.shape) for l in m.layers if isinstance(l, keras.layers.InputLayer)]

In [39]:
m = model_from_json(open("model_files/model.json", "r").read())

In [36]:
# all the input layers of the network
input_layers(m)

[(u'input_1', TensorShape([Dimension(None), Dimension(1002), Dimension(6)])),
 (u'input_2', TensorShape([Dimension(None), Dimension(1002), Dimension(6)])),
 (u'input_3', TensorShape([Dimension(None), Dimension(14)]))]

First two are from the sequence + two more dimensions:
- Unique35 score
- DNA accessibility mapping

In [40]:
# list all the features of the model
!cat feature.txt

bigwig
meta
gencode


In [46]:
# get all the bigwig tracks
!cat bigwig.txt

Unique35
DGF


In [47]:
# list all the PC components used
!cat meta.txt

GEPC1
GEPC2
GEPC3
GEPC4
GEPC5
GEPC6
GEPC7
GEPC8


In [48]:
# number of features
!wc -l meta.txt

8 meta.txt


In [ ]:
# -> the input is fixed 

In [55]:
ls ../../../data/PC-3/

bigwig.txt  meta.txt  PC-3.1x.bw  wgEncodeDukeMapabilityUniqueness35bp.bigWig@


In [56]:
cat ../../../data/PC-3/bigwig.txt

wgEncodeDukeMapabilityUniqueness35bp.bigWig	Unique35
PC-3.1x.bw	DGF


In [57]:
ls ../../../data/PC-3/

bigwig.txt  meta.txt  PC-3.1x.bw  wgEncodeDukeMapabilityUniqueness35bp.bigWig@


In [58]:
cat ../../../data/PC-3/bigwig.txt

wgEncodeDukeMapabilityUniqueness35bp.bigWig	Unique35
PC-3.1x.bw	DGF


In [59]:
# all the features for the additional things
# np.array([cpg.count, cds.count, intron.count, promoter.count, utr5.count, utr3.count], dtype=bool)

In [61]:
# gencode feature names
gencode_features = ['cpg', 'cds', 'intron', 'promoter', 'utr5', 'utr3']

## Plan how to write a dataloader

- write the bigwig part
  - args:
    - `interval_file` - standard bed file
    - `fasta_file` - reference genome
    - `dnase_file` - DNase bigwig
    - `cell_type` (default=None) - cell type as a string.
    - `RNAseq_PC_file` (default=None) - file path to a RNAseq PC file computed by https://github.com/davidaknowles/tf_net/blob/master/gene_expression_pca.R. See https://github.com/uci-cbcl/FactorNet/blob/master/data/README.md.
    - `mappability_file` (None)- USCS mappability track - http://hgdownload.cse.ucsc.edu/goldenpath/hg19/encodeDCC/wgEncodeMapability/wgEncodeDukeMapabilityUniqueness35bp.bigWig
      - by deafult, provide this file with the dataloader, download in background
    - `GENCODE_files` (default=None)
      - path to the already pre-processed gencode files directory to compute the gencode features
  - if each of those is none, exclude them from the analysis
- after you have this, write 4 different dataloaders and re-use them accross models
  - seq + dnase
  - seq + dnase + gencode
  - seq + dnase + PC
  - seq + dnase + gencode + PC

## TODO

- [ ] have the function get_file as in Keras
  - for now just use Keras
- [x] write the big-wig part of the dataloader
  - [x] write a small test set with the existing data

- [ ] setup 4 different dataloaders
  - [ ] maybe use jinja-templating to avoid sharing?
- setup a shared folder holding the shared files

## Generate example bigWig

In [1]:
import pyBigWig

In [2]:
mbw_fname = '../../../resources/wgEncodeDukeMapabilityUniqueness35bp.bigWig'

In [3]:
bw_fname = '../../../data/PC-3/PC-3.1x.bw'

In [4]:
bw = pyBigWig.open(bw_fname)

In [5]:
mbw = pyBigWig.open(mbw_fname)

In [6]:
bw.chroms()

{'chr1': 249250621L,
 'chr10': 135534747L,
 'chr11': 135006516L,
 'chr12': 133851895L,
 'chr13': 115169878L,
 'chr14': 107349540L,
 'chr15': 102531392L,
 'chr16': 90354753L,
 'chr17': 81195210L,
 'chr18': 78077248L,
 'chr19': 59128983L,
 'chr2': 243199373L,
 'chr20': 63025520L,
 'chr21': 48129895L,
 'chr22': 51304566L,
 'chr3': 198022430L,
 'chr4': 191154276L,
 'chr5': 180915260L,
 'chr6': 171115067L,
 'chr7': 159138663L,
 'chr8': 146364022L,
 'chr9': 141213431L,
 'chrM': 16571L,
 'chrX': 155270560L,
 'chrY': 59373566L}

In [8]:
#bw.values("chr1", 10000, 20000)

In [9]:
#mbw.values("chr1", 10000, 20000)

In [ ]:
# create an example bigwig file

In [10]:
ls

bigwig.txt         dataloader.pyc       feature.txt   other_files/
chip.txt           dataloader.yaml      model_files/  tasks.txt
dataloader_files/  example_files/       model.yaml
dataloader.py      explore_model.ipynb  new.bw


In [11]:
import pyBigWig

In [12]:
cd ../../

/srv/scratch/avsec/workspace/FactorNet/kipoi


In [13]:
nbw = pyBigWig.open("template/example_files/dnase_synth.chr22.bw", "w")

In [16]:
import six

In [17]:
nbw.addHeader([(k,v) for k,v in six.iteritems(bw.chroms())])

In [21]:
nbw.addEntries("chr22", [1100], values=[.5], span=10000)

In [22]:
nbw.close()

In [23]:
!du -sh template/example_files/*

4.0K	template/example_files/dnase_synth.chr22.bw
49M	template/example_files/hg38_chr22.fa
4.0K	template/example_files/intervals.bed


In [3]:
ls

bigwig.txt         dataloader.pyc       feature.txt   other_files/
chip.txt           dataloader.yaml      model_files/  tasks.txt
dataloader_files/  example_files/       model.yaml
dataloader.py      explore_model.ipynb  new.bw
